In [31]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

**Reboot: Box-Plots for Education competition**

**Data importation : training data**

In [2]:
train_data = pd.read_csv('../data/TrainingData.csv', index_col=[0])

In [3]:
train_data.head(5)

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,...,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,CONTRACTOR SERVICES,...,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,...,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,...,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,...,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


In [4]:
X = train_data.loc[:, 'Object_Description':]
X.head(5)

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,NaN,NaN,NaN,Teacher-Elementary,NaN,NaN,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,CONTRACTOR SERVICES,BOND EXPENDITURES,BUILDING FUND,(blank),Regular,NaN,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Personal Services - Teachers,NaN,NaN,TCHER 2ND GRADE,NaN,Regular Instruction,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,EMPLOYEE BENEFITS,TEACHER SUBS,GENERAL FUND,"Teacher, Short Term Sub",Regular,NaN,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,TEACHER COVERAGE FOR TEACHER,TEACHER SUBS,GENERAL FUND,"Teacher, Secondary (High)",Alternative,NaN,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


In [5]:
y = train_data.loc[:, :'Operating_Status']
y.head(5)

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating


**Data importation : testing data**

In [6]:
X_test = pd.read_csv('../data/TestData.csv', index_col=[0])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
X_test.head(5)

,Object_Description,Program_Description,SubFund_Description,Job_Title_Description,Facility_or_Department,Sub_Object_Description,Location_Description,FTE,Function_Description,Position_Extra,Text_4,Total,Text_2,Text_3,Fund_Description,Text_1
180042,Student Meals/Room/Other,Basic Educational Services,NaN,NaN,NaN,Line Item that is paid with Campus' money,School,NaN,Instruction,NaN,NaN,3999.910000,NaN,NaN,General Fund,NaN
28872,Extra Duty/Signing Bonus Pay,Undistributed,NaN,CHEERLEADER DIR,NaN,General,School,NaN,Cocurricular & Extra Curricular Activities,NaN,NaN,3447.320213,NaN,NaN,General Fund,NaN
186915,Professional Salaries,Bilingual Education,NaN,T-EL 1ST BIL,NaN,General,School,1.0,Instruction,NaN,NaN,52738.780869,NaN,NaN,General Fund,NaN
412396,Professional Salaries,Bilingual Education,NaN,T-EL 2ND BIL,NaN,General,School,1.0,Instruction,NaN,NaN,69729.263191,NaN,NaN,General Fund,NaN
427740,Salaries for Support Personnel,Undistributed,NaN,CLERK III- SCH,NaN,General,School,1.0,School Leadership,NaN,NaN,29492.834215,NaN,NaN,General Fund,NaN


**Test data column ordering must be equal to Training data**

In [8]:
X_test1 = pd.DataFrame(X_test,columns=X.columns)

In [9]:
X_test1.head(5)

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
180042,Student Meals/Room/Other,NaN,NaN,NaN,NaN,NaN,Line Item that is paid with Campus' money,School,NaN,Instruction,NaN,NaN,3999.910000,Basic Educational Services,General Fund,NaN
28872,Extra Duty/Signing Bonus Pay,NaN,NaN,CHEERLEADER DIR,NaN,NaN,General,School,NaN,Cocurricular & Extra Curricular Activities,NaN,NaN,3447.320213,Undistributed,General Fund,NaN
186915,Professional Salaries,NaN,NaN,T-EL 1ST BIL,NaN,NaN,General,School,1.0,Instruction,NaN,NaN,52738.780869,Bilingual Education,General Fund,NaN
412396,Professional Salaries,NaN,NaN,T-EL 2ND BIL,NaN,NaN,General,School,1.0,Instruction,NaN,NaN,69729.263191,Bilingual Education,General Fund,NaN
427740,Salaries for Support Personnel,NaN,NaN,CLERK III- SCH,NaN,NaN,General,School,1.0,School Leadership,NaN,NaN,29492.834215,Undistributed,General Fund,NaN


**Inspect Data Types**

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400277 entries, 134338 to 415831
Data columns (total 16 columns):
Object_Description        375493 non-null object
Text_2                    88217 non-null object
SubFund_Description       306855 non-null object
Job_Title_Description     292743 non-null object
Text_3                    109152 non-null object
Text_4                    53746 non-null object
Sub_Object_Description    91603 non-null object
Location_Description      162054 non-null object
FTE                       126071 non-null float64
Function_Description      342195 non-null object
Facility_or_Department    53886 non-null object
Position_Extra            264764 non-null object
Total                     395722 non-null float64
Program_Description       304660 non-null object
Fund_Description          202877 non-null object
Text_1                    292285 non-null object
dtypes: float64(2), object(14)
memory usage: 51.9+ MB


**Seperate Numerical and Text Features**

In [11]:
num_cols = X.select_dtypes(include='number').columns.to_list()
text_cols = X.select_dtypes(exclude='number').columns.to_list()
text_cols

['Object_Description',
 'Text_2',
 'SubFund_Description',
 'Job_Title_Description',
 'Text_3',
 'Text_4',
 'Sub_Object_Description',
 'Location_Description',
 'Function_Description',
 'Facility_or_Department',
 'Position_Extra',
 'Program_Description',
 'Fund_Description',
 'Text_1']

In [12]:
X_test1.shape

(50064, 16)

**Create csv submission file**

In [13]:
def create_csv_submission_file(model, X_test, y_train):
    # Make prediction
    y_pred = model.predict_proba(X_test)
    
    # Make list of column names
    pc_pairs = zip(y_train.columns, model.named_steps.classifier.classes_)
    columns = []
    for parent, children in pc_pairs:
        for child in children:
            columns.append(f'{parent}__{child}')
            
    # Put predictions in DataFrame
    df = pd.DataFrame(np.concatenate(y_pred, axis=1),
                      index=X_test.index,
                      columns=columns)
    df = df[sorted(df.columns)]
    
    date = pd.Timestamp.now().strftime(format='%Y-%m-%d_%H-%M_')
    df.to_csv(f'../predictions/{date}submission.csv', index=True, header=True)

**Custom transformer to join all the text in the text columns**

In [14]:
class Concatenator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        results = []
        for row in X:
            results.append(' '.join([w for w in row if w]))
        return results

In [15]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

text_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('concatenate', Concatenator()),
    ('vectorizer', HashingVectorizer(lowercase=True))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('num_pipe', num_pipe, num_cols),
                           ('text_pipe', text_pipe, text_cols)
                       ])

model = Pipeline([
    ('ct', ct),
    ('classifier', DecisionTreeClassifier(max_depth=3))
])


In [16]:
model.fit(X,y);

In [17]:
create_csv_submission_file(model, X_test1, y)

In [18]:
y_pred = model.predict_proba(X_test1)

In [19]:
pd.DataFrame(np.concatenate(y_pred, axis=1),index = X_test1.index).head()

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,103
180042,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
28872,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
186915,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
412396,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
427740,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633


In [20]:
pd.DataFrame(np.hstack(y_pred),index = X_test1.index).head()

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,103
180042,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
28872,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
186915,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
412396,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
427740,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633


In [21]:
y.columns

Index(['Function', 'Use', 'Sharing', 'Reporting', 'Student_Type',
       'Position_Type', 'Object_Type', 'Pre_K', 'Operating_Status'],
      dtype='object')

In [22]:
model.named_steps.classifier.classes_

[array(['Aides Compensation', 'Career & Academic Counseling',
        'Communications', 'Curriculum Development',
        'Data Processing & Information Services',
        'Development & Fundraising', 'Enrichment',
        'Extended Time & Tutoring', 'Facilities & Maintenance',
        'Facilities Planning',
        'Finance, Budget, Purchasing & Distribution', 'Food Services',
        'Governance', 'Human Resources',
        'Instructional Materials & Supplies', 'Insurance', 'Legal',
        'Library & Media', 'NO_LABEL', 'Other Compensation',
        'Other Non-Compensation', 'Parent & Community Relations',
        'Physical Health & Services', 'Professional Development',
        'Recruitment', 'Research & Accountability',
        'School Administration', 'School Supervision', 'Security & Safety',
        'Social & Emotional',
        'Special Population Program Management & Support',
        'Student Assignment', 'Student Transportation',
        'Substitute Compensation', 'Teacher 

In [23]:
# Make list of column names
pc_pairs = zip(y.columns, model.named_steps.classifier.classes_)
columns = []
for parent, children in pc_pairs:
    for child in children:
        columns.append(f'{parent}__{child}')

In [24]:
columns

['Function__Aides Compensation',
 'Function__Career & Academic Counseling',
 'Function__Communications',
 'Function__Curriculum Development',
 'Function__Data Processing & Information Services',
 'Function__Development & Fundraising',
 'Function__Enrichment',
 'Function__Extended Time & Tutoring',
 'Function__Facilities & Maintenance',
 'Function__Facilities Planning',
 'Function__Finance, Budget, Purchasing & Distribution',
 'Function__Food Services',
 'Function__Governance',
 'Function__Human Resources',
 'Function__Instructional Materials & Supplies',
 'Function__Insurance',
 'Function__Legal',
 'Function__Library & Media',
 'Function__NO_LABEL',
 'Function__Other Compensation',
 'Function__Other Non-Compensation',
 'Function__Parent & Community Relations',
 'Function__Physical Health & Services',
 'Function__Professional Development',
 'Function__Recruitment',
 'Function__Research & Accountability',
 'Function__School Administration',
 'Function__School Supervision',
 'Function__Se

In [25]:
pd.DataFrame(np.concatenate(y_pred, axis=1),
                      index=X_test1.index,
                      columns=columns).head()

,Function__Aides Compensation,Function__Career & Academic Counseling,Function__Communications,Function__Curriculum Development,Function__Data Processing & Information Services,Function__Development & Fundraising,Function__Enrichment,Function__Extended Time & Tutoring,Function__Facilities & Maintenance,Function__Facilities Planning,...,Object_Type__Rent/Utilities,Object_Type__Substitute Compensation,Object_Type__Supplies/Materials,Object_Type__Travel & Conferences,Pre_K__NO_LABEL,Pre_K__Non PreK,Pre_K__PreK,Operating_Status__Non-Operating,"Operating_Status__Operating, Not PreK-12",Operating_Status__PreK-12 Operating
180042,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
28872,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
186915,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
412396,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
427740,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633


In [26]:
pd.DataFrame(np.hstack(y_pred),
             index = X_test1.index,
             columns=columns).head()

,Function__Aides Compensation,Function__Career & Academic Counseling,Function__Communications,Function__Curriculum Development,Function__Data Processing & Information Services,Function__Development & Fundraising,Function__Enrichment,Function__Extended Time & Tutoring,Function__Facilities & Maintenance,Function__Facilities Planning,...,Object_Type__Rent/Utilities,Object_Type__Substitute Compensation,Object_Type__Supplies/Materials,Object_Type__Travel & Conferences,Pre_K__NO_LABEL,Pre_K__Non PreK,Pre_K__PreK,Operating_Status__Non-Operating,"Operating_Status__Operating, Not PreK-12",Operating_Status__PreK-12 Operating
180042,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
28872,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
186915,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
412396,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633
427740,0.098446,0.013723,0.001562,0.028246,0.014704,0.000432,0.038973,0.00892,0.103015,0.000155,...,0.017002,0.010295,0.1548,0.025165,0.781031,0.184784,0.034185,0.130008,0.026359,0.843633


In [27]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

text_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('concatenate', Concatenator()),
    ('vectorizer', HashingVectorizer(lowercase=True))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[
                           ('num_pipe', num_pipe, num_cols),
                           ('text_pipe', text_pipe, text_cols)
                       ])

randommodel = Pipeline([
    ('ct', ct),
    ('classifier', RandomForestClassifier(n_jobs=-1,n_estimators=100,verbose=1))
])

In [28]:
randommodel.fit(X,y);

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 271.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 619.0min finished


In [29]:
create_csv_submission_file(randommodel, X_test1, y)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  4.7min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  5.5min finished
